In [ ]:
import os
import sys
sys.path.append("/mnt/code")

In [ ]:
# The following import and function call are the only additions to code required
# to automatically log metrics and parameters to MLflow.
import mlflow
from domino_mlflow_utils.mlflow_utilities import *
mlflow_utils = DominoMLflowUtilities()

In [ ]:
import os
## Apply Autologging for Tensorflow
mlflow.tensorflow.autolog()
experiment_name= os.environ['DOMINO_PROJECT_NAME'] + "-mlflow_tensorboard"
mlflow_utils.init(experiment_name)

In [ ]:
import os
# Don't use GPU
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
"""Trains and evaluate a simple MLP
on the Reuters newswire topic classification task.
"""
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import reuters
from tensorflow.keras.layers import Activation, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer


mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
      return tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
      ])

model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from tbparse import SummaryReader
#reader = SummaryReader(log_dir, extra_columns={'dir_name'})
#df = reader.tensors
tmpdirs = {}
tmpdirs['tensorflow'] = '/tmp/logs-4'
log_dir = '/tmp/logs-4'
print(log_dir)
writer = tf.summary.create_file_writer('/tmp/logs-4')
writer.set_as_default()

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='/tmp/logs-4',histogram_freq=1,write_graph=False, write_images=False)
model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])



## Download Tensorboard Logs

The next cell demonstrates how to fetch the tensorboard logs for any run for which these logs have been registered.

A local tensorboard instance can be used to render these logs. The benefit of this approach is that it can be used to register tensorboard logs for any experiment run and can be downloaded when needed for review. They do not have to be stored in your workspace or repo.

You can go to the run artifacts UI to verify the detailed logs.

WANDB shows the following set of additional metrics as compared to MLFLOW. User defined code could provide the same metrics in MLFLOW
![WANDB TENSORBOARD METRICS](./images/wandb_tensorflow_metrics.png)

In [ ]:
r = mlflow.active_run()
run_id=r.info.run_id
home_dir = os.path.expanduser('~')
download_path=f'{home_dir}/{run_id}'
mlflow.artifacts.download_artifacts(run_id=run_id,dst_path=download_path)
print(f"The artifacts for {run_id} will be downloaded to the folder {download_path}")
os.listdir(download_path)

In [ ]:


mlflow_utils.finish()


In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from collections import defaultdict, namedtuple
from typing import List
import tensorflow as tf
from tensorflow.core.util import event_pb2 

TensorBoardImage = namedtuple("TensorBoardImage", ["topic", "image", "cnt"])

data = []
def extract_images_from_event(event_filename: str, image_tags: List[str]):
    step = 0
    my_list = {}
    
    
    topic_counter = defaultdict(lambda: 0)

    serialized_examples = tf.data.TFRecordDataset(event_filename)
    for serialized_example in serialized_examples:
        wtimes,steps,actions = [],[],[]
        #print(dir(serialized_example))
        #print(serialized_example.value_index)
        event = event_pb2.Event.FromString(serialized_example.numpy())
        #print(type(event.summary))
        wtime   = event.wall_time
        step    = event.step
        
        for v in event.summary.value:
            #print(v.tag)
            if v.tag in image_tags:                                
                if v.HasField('histo'):  # event for images using tensor field

                    #print(v.tag)                   
                    hist = v.histo
                    #print(hist)
                    wtimes += [wtime]*int(v.histo.num)
                    steps  += [step] *int(v.histo.num)
                    #print(steps)
                    for num,val in zip(v.histo.bucket,v.histo.bucket_limit):
                        actions += [val] *int(num)
                    my_list[step]={'wtimes':wtimes,'steps':steps,'actions':actions}
                    #print(wtimes)
                    step = step + 1
        
    '''
    for k,v in my_list.items():
        print(k)
        print(v)
    N_points = 10
    n_bins = 5
    '''

    return my_list

def plot_tb(name,my_list):
    # Generate two normal distributions
    x0 = my_list[0]['actions']
    x1 = my_list[1]['actions']
    x2 = my_list[2]['actions']
    x3 = my_list[3]['actions']
    x4 = my_list[4]['actions']
    d1 = {'Step0': x0, 'Step1': x1, 'Step2': x2, 'Step3': x3, 'Step4': x4}

    df = pd.DataFrame(data=d1)

    plt.figure(figsize=(12,5))
    plt.xlabel(name)

    ax1 = df['Step0'].plot(color='blue', grid=True, label='Step0')
    ax2 = df['Step1'].plot(color='green', grid=True,  label='Step1')
    ax2 = df['Step2'].plot(color='red', grid=True,  label='Step2')
    ax2 = df['Step3'].plot(color='cyan', grid=True,  label='Step3')
    ax2 = df['Step4'].plot(color='magenta', grid=True,  label='Step4')


    h1, l1 = ax1.get_legend_handles_labels()
    #h2, l2 = ax2.get_legend_handles_labels()


    plt.legend(h1, l1, loc=2,fancybox=True, framealpha=0.5)
    plt.show()

                  
 


In [ ]:
p='/tmp/logs-4/train/events.out.tfevents.1709028637.run-65ddada83f0b1c0e070b0856-lj2n4.832.142.v2'
tags = ['dense/bias_0','dense/kernel_0','dense_1/bias_0','dense_1/kernel_0']
my_list = extract_images_from_event(p,[tags[0]])
plot_tb(tags[0],my_list)

my_list = extract_images_from_event(p,[tags[1]])
plot_tb(tags[1],my_list)

my_list = extract_images_from_event(p,[tags[2]])
plot_tb(tags[2],my_list)

my_list = extract_images_from_event(p,[tags[3]])
plot_tb(tags[3],my_list)



In [ ]:
#!jupyter nbconvert --clear-output --inplace notebooks/basic/*.ipynb